
# Case Study 5: Bayesian Neural Network

Adapted from https://num.pyro.ai/en/stable/examples/bnn.html , we first see the NumPyro implementation and then SOGA.


In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *
from time import time

torch.set_default_dtype(torch.float64)

In [ ]:
import argparse
import os
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import jax
from jax import vmap
import jax.numpy as jnp
import jax.random as random

import numpyro
from numpyro import handlers
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

matplotlib.use("Agg")  # noqa: E402


# the non-linearity we use in our neural network
def nonlin(x):
    return jax.nn.relu(x)


# a two-layer bayesian neural network with computational flow
# given by D_X => D_H => D_H => D_Y where D_H is the number of
# hidden units. (note we indicate tensor dimensions in the comments)
def model(X, Y, D_H, D_Y=1):
    N, D_X = X.shape

    # sample first layer (we put unit normal priors on all weights)
    w1 = numpyro.sample("w1", dist.Normal(jnp.zeros((D_X, D_H)), jnp.ones((D_X, D_H))))
    assert w1.shape == (D_X, D_H)
    z1 = nonlin(jnp.matmul(X, w1))  # <= first layer of activations
    assert z1.shape == (N, D_H)

    # sample second layer
    w2 = numpyro.sample("w2", dist.Normal(jnp.zeros((D_H, D_H)), jnp.ones((D_H, D_H))))
    assert w2.shape == (D_H, D_H)
    z2 = nonlin(jnp.matmul(z1, w2))  # <= second layer of activations
    assert z2.shape == (N, D_H)

    # sample final layer of weights and neural network output
    w3 = numpyro.sample("w3", dist.Normal(jnp.zeros((D_H, D_Y)), jnp.ones((D_H, D_Y))))
    assert w3.shape == (D_H, D_Y)
    z3 = jnp.matmul(z2, w3)  # <= output of the neural network
    assert z3.shape == (N, D_Y)

    if Y is not None:
        assert z3.shape == Y.shape

    # we put a prior on the observation noise
    prec_obs = numpyro.sample("prec_obs", dist.Normal(0., 1.0))  #Originally Gamma(3.0, 1.0)
    sigma_obs = 1.0 / jnp.sqrt(prec_obs)

    # observe data
    with numpyro.plate("data", N):
        # note we use to_event(1) because each observation has shape (1,)
        numpyro.sample("Y", dist.Normal(z3, sigma_obs).to_event(1), obs=Y)


# helper function for HMC inference
def run_inference(model, rng_key, X, Y, D_H):
    start = time.time()
    kernel = NUTS(model)
    mcmc = MCMC(
        kernel,
        num_warmup=1000,
        num_samples=2000,
        num_chains=1,
        progress_bar=False if "NUMPYRO_SPHINXBUILD" in os.environ else True,
    )
    mcmc.run(rng_key, X, Y, D_H)
    mcmc.print_summary()
    print("\nMCMC elapsed time:", time.time() - start)
    return mcmc.get_samples()


# helper function for prediction
def predict(model, rng_key, samples, X, D_H):
    model = handlers.substitute(handlers.seed(model, rng_key), samples)
    # note that Y will be sampled in the model because we pass Y=None here
    model_trace = handlers.trace(model).get_trace(X=X, Y=None, D_H=D_H)
    return model_trace["Y"]["value"]


# create artificial regression dataset
def get_data(N=20, D_X=3, sigma_obs=0.05, N_test=500):
    D_Y = 1  # create 1d outputs
    np.random.seed(0)
    X = jnp.linspace(-1, 1, N)
    X = jnp.power(X[:, np.newaxis], jnp.arange(D_X))
    W = 0.5 * np.random.randn(D_X)
    Y = jnp.dot(X, W) + 0.5 * jnp.power(0.5 + X[:, 1], 2.0) * jnp.sin(4.0 * X[:, 1])
    Y += sigma_obs * np.random.randn(N)
    Y = Y[:, np.newaxis]
    Y -= jnp.mean(Y)
    Y /= jnp.std(Y)

    assert X.shape == (N, D_X)
    assert Y.shape == (N, D_Y)

    X_test = jnp.linspace(-1.3, 1.3, N_test)
    X_test = jnp.power(X_test[:, np.newaxis], jnp.arange(D_X))

    return X, Y, X_test


args = [10, 2, 2]
N, D_X, D_H = args
X, Y, X_test = get_data(N=N, D_X=D_X)

# do inference
rng_key, rng_key_predict = random.split(random.PRNGKey(0))
samples = run_inference(model, rng_key, X, Y, D_H)

# predict Y_test at inputs X_test
vmap_args = (
    samples,
    random.split(rng_key_predict, 2000 * 1),
)
predictions = vmap(
    lambda samples, rng_key: predict(model, rng_key, samples, X_test, D_H)
)(*vmap_args)
predictions = predictions[..., 0]

# compute mean prediction and confidence interval around median
mean_prediction = jnp.mean(predictions, axis=0)
percentiles = np.percentile(predictions, [5.0, 95.0], axis=0)

# make plots
fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

# plot training data
ax.plot(X[:, 1], Y[:, 0], "kx")
# plot 90% confidence level of predictions
ax.fill_between(
    X_test[:, 1], percentiles[0, :], percentiles[1, :], color="lightblue"
)
# plot mean prediction
ax.plot(X_test[:, 1], mean_prediction, "blue", ls="solid", lw=2.0)
ax.set(xlabel="X", ylabel="Y", title="Mean predictions with 90% CI")

plt.savefig("bnn_plot.pdf")

sample: 100%|██████████| 3000/3000 [00:01<00:00, 2460.02it/s, 31 steps of size 1.12e-01. acc. prob=0.81]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  prec_obs      1.01      0.29      0.99      0.53      1.48   1459.66      1.00
   w1[0,0]     -0.18      1.01     -0.24     -1.84      1.43   1153.05      1.00
   w1[0,1]     -0.18      0.97     -0.18     -1.74      1.52    628.04      1.00
   w1[1,0]     -0.02      0.95     -0.02     -1.62      1.44    841.15      1.00
   w1[1,1]     -0.03      0.93     -0.04     -1.40      1.64    903.38      1.00
   w2[0,0]     -0.14      0.96     -0.13     -1.80      1.38    966.95      1.00
   w2[0,1]     -0.16      1.01     -0.18     -1.71      1.58    938.67      1.00
   w2[1,0]     -0.10      0.92     -0.10     -1.58      1.49   1109.30      1.00
   w2[1,1]     -0.09      0.99     -0.10     -1.71      1.54    711.64      1.00
   w3[0,0]     -0.10      0.92     -0.10     -1.53      1.53    657.53      1.00
   w3[1,0]     -0.11      0.96     -0.10     -1.72      1.46   1207.36      1.00

Number of divergences: 29


In [3]:
print((X[:,1]).tolist())
print(X[:,1].shape)

[-1.0, -0.8947368860244751, -0.7894736528396606, -0.6842105388641357, -0.5789473056793213, -0.4736841917037964, -0.3684210777282715, -0.2631578743457794, -0.15789473056793213, -0.05263158679008484, 0.05263161659240723, 0.15789473056793213, 0.26315784454345703, 0.3684210777282715, 0.4736841917037964, 0.5789474248886108, 0.6842105388641357, 0.7894736528396606, 0.8947368860244751, 1.0]
(20,)


In [4]:
def optimize(params_dict, loss_function, y, cfg, steps=500):
    optimizer = torch.optim.Adam([params_dict[key] for key in params_dict.keys()], lr=1)

    total_start = time()

    for i in range(steps):

        optimizer.zero_grad()  # Reset gradients
        
        # loss
        current_dist = start_SOGA(cfg, params_dict)

        loss = loss_function(y, current_dist)

        # Backpropagate
        loss.backward(retain_graph=True)
        
        optimizer.step()

        # Print progress
        if i % 10 == 0:
            out = ''
            for key in params_dict.keys():
                out = out + key + ': ' + str(params_dict[key].item()) + ' '
            out = out + f" loss: {loss.item()}"
            print(out)

    total_end = time()

    print('Optimization performed in ', round(total_end-total_start, 3))

In [5]:
def mean_squared_error(y_true, dist):
    return torch.mean((y_true - dist.gm.mean()) ** 2)

def mean_squared_error_bayes(y_true, dist):
    #This works for the means but of course not for the variances
    return torch.mean((y_true - dist.gm.mean()[:-2]) ** 2)

def neg_log_likelihood(y_true, dist):
    #Calculate the log-likelihood of the data given the distribution
    neg_log_likelihood = 0
    for i in range(len(dist.gm.mean())-2):
        neg_log_likelihood -= torch.log(dist.gm.marg_pdf(y_true[i].unsqueeze(0), i))
    return neg_log_likelihood

In [6]:
print((X[:,1]).tolist())
print(X[:,1].shape)

[-1.0, -0.8947368860244751, -0.7894736528396606, -0.6842105388641357, -0.5789473056793213, -0.4736841917037964, -0.3684210777282715, -0.2631578743457794, -0.15789473056793213, -0.05263158679008484, 0.05263161659240723, 0.15789473056793213, 0.26315784454345703, 0.3684210777282715, 0.4736841917037964, 0.5789474248886108, 0.6842105388641357, 0.7894736528396606, 0.8947368860244751, 1.0]
(20,)


In [7]:
compiledFile=compile2SOGA('../programs/SOGA/Optimization/Case Studies/bnn3.soga')
cfg = produce_cfg(compiledFile)

#pars = {'mu100':0., 'sigma100':1., 'mu101':0., 'sigma101':1.,'mu110':0., 'sigma110':1.,'mu111':0., 'sigma111':1.,'mu200':0., 'sigma200':1.,
        #'mu201':0., 'sigma201':1.,'mu210':0., 'sigma210':1.,'mu211':0., 'sigma211':1.,'mu300':0., 'sigma300':1.,'mu310':0., 'sigma310':1.,}


#pars = {'mu100':0., 'sigma100':1., 'mu101':0., 'sigma101':1.,'mu110':0., 'sigma110':1.,'mu111':0., 'sigma111':1.,'mu300':0., 'sigma300':1.,'mu310':0., 'sigma310':1.}
params_dict = {}
#for key, value in pars.items():
    #params_dict[key] = torch.tensor(value, requires_grad=True)    

output_dist = start_SOGA(cfg) #params_dict

#optimize(params_dict, neg_log_likelihood, Y, cfg, steps=20)

#predictive mean
#y_pred = params_dict['muw'].detach().numpy()*X.detach().numpy()+params_dict['mub'].detach().numpy()

#predictive variance
#sigma_y_pred = np.sqrt(params_dict['sigmay'].detach().numpy()**2 + (X.detach().numpy()*params_dict['sigmaw'].detach().numpy())**2 + params_dict['sigmab'].detach().numpy()**2)



Entering EntryNode<>
1  components


Entering StateNode<state0,None,w100=gm([1.],[0.],[1.])>
1  components


Entering StateNode<state1,None,w101=gm([1.],[0.],[1.])>
1  components


Entering StateNode<state2,None,w110=gm([1.],[0.],[1.])>
1  components


Entering StateNode<state3,None,w111=gm([1.],[0.],[1.])>
1  components


Entering StateNode<state4,None,w300=gm([1.],[0.],[1.])>
1  components


Entering StateNode<state5,None,w310=gm([1.],[0.],[1.])>
1  components


Entering LoopNode<loop0>
1  components


Entering StateNode<state6,True,a10[i]=w100*X[i]>
1  components


Entering StateNode<state7,None,a10[i]=a10[i]+w110>
1  components


Entering StateNode<state8,None,a11[i]=w101*X[i]>
1  components


Entering StateNode<state9,None,a11[i]=a11[i]+w111>
1  components


Entering TestNode<test0,a10[i]<0.0>
1  components


Entering StateNode<state10,True,a10[i]=gm([1.0],[0.000000],[0.010000])>
1  components


Entering StateNode<state11,False,skip>
1  components


Entering MergeNode<merge0>


En

In [8]:
output_dist

Dist<['y[0]', 'y[1]', 'y[2]', 'y[3]', 'y[4]', 'y[5]', 'y[6]', 'y[7]', 'y[8]', 'y[9]', 'a10[0]', 'a10[1]', 'a10[2]', 'a10[3]', 'a10[4]', 'a10[5]', 'a10[6]', 'a10[7]', 'a10[8]', 'a10[9]', 'a11[0]', 'a11[1]', 'a11[2]', 'a11[3]', 'a11[4]', 'a11[5]', 'a11[6]', 'a11[7]', 'a11[8]', 'a11[9]', 'w100', 'w101', 'w110', 'w111', 'w300', 'w310'],pi: tensor([[0.3573],
        [0.0293],
        [0.0037],
        ...,
        [0.0037],
        [0.0293],
        [0.3573]])
mu: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
sigma: tensor([[[1.0000e-05, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-05, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 1.0000e-05,  ..., 0.0000e+00,
          0.0